In [1]:
# -*- coding: utf-8 -*-

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import folium

     |████████████████████████████████| 972kB 8.1MB/s 
     |████████████████████████████████| 6.5MB 24.7MB/s 
     |████████████████████████████████| 14.8MB 270kB/s 


# 데이터 전처리

## 1. 전유부, 표제부

In [3]:
전유부 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/1.세종시_전유부.csv', encoding= 'CP949')
표제부 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/2.세종시_표제부.csv', encoding= 'CP949')

전유부.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,특수지명,블록,로트,동명칭,호명칭,층구분코드명,층번호
0,1,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,616,지상,6
1,2,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,624,지상,6
2,3,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,523,지상,5
3,4,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,314,지상,3
4,5,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,101,지상,1


In [4]:
표제부.head()

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,NaN
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,1동,576,강파이프구조,동.식물관련시설,0,1999.0,NaN
2,3,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,2동,336,강파이프구조,동.식물관련시설,0,20041223.0,NaN
3,4,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,3동,127.19,강파이프구조,창고시설,0,19950726.0,NaN
4,5,세종특별자치시 반곡동 180-1번지,36110,10100,1,180,1,일반,솔빛숲유치원,NaN,2697.74,철근콘크리트구조,교육연구시설,0,20190228.0,NaN


In [5]:
전유부.describe(include='all')

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,특수지명,블록,로트,동명칭,호명칭,층구분코드명,층번호
count,40000.00000,40000,40000.0,40000.00000,40000.000000,40000.000000,40000.000000,40000,39924,24967,24788,1065,39235,40000,40000,40000.000000
unique,NaN,132,NaN,NaN,NaN,NaN,NaN,1,124,19,40,6,309,2001,2,NaN
top,NaN,세종특별자치시 소담동 블록 행정중심복합도시 3-3생활권지구 M6블록블록,NaN,NaN,NaN,NaN,NaN,집합,첫마을아파트,행정중심복합도시 3-3생활권,M6블록,M4블록,주건축물제1동,301,지상,NaN
freq,NaN,1540,NaN,NaN,NaN,NaN,NaN,40000,6749,6241,3534,674,1001,563,39866,NaN
mean,20000.50000,NaN,36110.0,10332.64500,2.248350,302.650125,6.976025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.875425
std,11547.14972,NaN,0.0,179.79881,0.968682,494.084730,75.754958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.979328
min,1.00000,NaN,36110.0,10100.00000,1.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,10000.75000,NaN,36110.0,10200.00000,1.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
50%,20000.50000,NaN,36110.0,10300.00000,3.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000
75%,30000.25000,NaN,36110.0,10400.00000,3.000000,713.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.000000


In [6]:
표제부.describe(include='all')

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
count,36729.000000,36729,36729.0,36729.000000,36729.000000,36729.000000,36729.000000,36729,7326,16491,36729,36720,36717,36729,3.551700e+04,1.0
unique,NaN,24484,NaN,NaN,NaN,NaN,NaN,2,2104,2224,20837,30,34,194,NaN,NaN
top,NaN,세종특별자치시 금남면 도남리 12-2번지,NaN,NaN,NaN,NaN,NaN,일반,1동,1동,0,철근콘크리트구조,단독주택,0,NaN,NaN
freq,NaN,70,NaN,NaN,NaN,NaN,NaN,32579,293,2647,374,9947,18021,30212,NaN,NaN
mean,18365.000000,NaN,36110.0,30262.805658,1.124234,310.429388,6.002968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.574662e+07,20130823.0
std,10602.893355,NaN,0.0,8545.648928,0.472338,306.729458,48.020151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.227412e+06,NaN
min,1.000000,NaN,36110.0,10100.000000,1.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,20130823.0
25%,9183.000000,NaN,36110.0,25032.000000,1.000000,90.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.980011e+07,20130823.0
50%,18365.000000,NaN,36110.0,34022.000000,1.000000,226.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.001091e+07,20130823.0
75%,27547.000000,NaN,36110.0,36028.000000,1.000000,436.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.014021e+07,20130823.0


### 변수 삭제

'번' + '지' = '대지위치' 이므로 변수 '번', '지' 삭제

In [7]:
전유부 = 전유부.drop(['번','지'], axis=1)
표제부 = 표제부.drop(['번','지'], axis=1)

동명칭, 호명칭, 층번호 삭제 - 임의 삭제. 시각화된 이유X

In [8]:
전유부 = 전유부.drop(['동명칭','호명칭','층번호'], axis=1)
표제부 = 표제부.drop('동명칭', axis=1)

In [9]:
전유부.isnull().sum()

순번             0
대지위치           0
시군구코드          0
법정동코드          0
대지구분코드         0
대장구분코드명        0
건물명           76
특수지명       15033
블록         15212
로트         38935
층구분코드명         0
dtype: int64

In [10]:
표제부.isnull().sum()

순번                 0
대지위치               0
시군구코드              0
법정동코드              0
대지구분코드             0
대장구분코드명            0
건물명            29403
연면적(㎡)             0
구조코드명              9
주용도코드명            12
세대수(세대)            0
사용승인일           1212
Unnamed: 15    36728
dtype: int64

전유부의 '로트' 변수와 표제부의 'Unnamed: 15' 변수는 거의 대부분이 결측이므로 변수 삭제

In [11]:
전유부 = 전유부.drop('로트', axis=1)
표제부 = 표제부.drop('Unnamed: 15', axis=1)

In [12]:
전유부.head()

,순번,대지위치,시군구코드,법정동코드,대지구분코드,대장구분코드명,건물명,특수지명,블록,층구분코드명
0,1,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,집합,세종펠리스,NaN,NaN,지상
1,2,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,집합,세종펠리스,NaN,NaN,지상
2,3,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,집합,세종펠리스,NaN,NaN,지상
3,4,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,집합,세종펠리스,NaN,NaN,지상
4,5,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,집합,세종펠리스,NaN,NaN,지상


In [13]:
표제부.head()

,순번,대지위치,시군구코드,법정동코드,대지구분코드,대장구분코드명,건물명,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,일반,NaN,135.24,벽돌구조,단독주택,0,1977.0
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,일반,NaN,576,강파이프구조,동.식물관련시설,0,1999.0
2,3,세종특별자치시 반곡동 146-79번지,36110,10100,1,일반,NaN,336,강파이프구조,동.식물관련시설,0,20041223.0
3,4,세종특별자치시 반곡동 146-79번지,36110,10100,1,일반,NaN,127.19,강파이프구조,창고시설,0,19950726.0
4,5,세종특별자치시 반곡동 180-1번지,36110,10100,1,일반,솔빛숲유치원,2697.74,철근콘크리트구조,교육연구시설,0,20190228.0


## 2. 매매 및 전월세 실거래가

### 2.1. 매매 실거래가

In [14]:
아파트_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/3.세종시_아파트(매매)_실거래가.csv', encoding= 'CP949')
연립다세대_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/4.세종시_연립다세대(매매)_실거래가.csv', encoding='CP949')
단독다가구_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/5.세종시_단독다가구(매매)_실거래가.csv', encoding='CP949')
오피스텔_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/6.세종시_오피스텔(매매)_실거래가.csv', encoding='CP949')
아파트_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/7.세종시_아파트(전월세)_실거래가.csv', encoding='CP949')
연립다세대_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/8.세종시_연립다세대(전월세)_실거래가.csv', encoding='CP949')
단독다가구_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/9.세종시_단독다가구(전월세)_실거래가.csv', encoding='CP949')
오피스텔_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/10.세종시_오피스텔(전월세)_실거래가.csv', encoding='CP949')

In [15]:
print(list(아파트_매매.columns))
print(list(연립다세대_매매.columns))
print(list(단독다가구_매매.columns))
print(list(오피스텔_매매.columns))
print(list(아파트_전월세.columns))
print(list(연립다세대_전월세.columns))
print(list(단독다가구_전월세.columns))
print(list(오피스텔_전월세.columns))

['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전용면적(㎡)', '대지권면적(㎡)', '거래금액(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '주택유형', '도로조건', '연면적(㎡)', '대지면적(㎡)', '거래금액(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '도로조건', '계약면적(㎡)', '전월세구분', '보증금(만원)', '월세(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']


매매 데이터의 공통 변수인 '계약년월','계약일','시군구','거래금액','건축년도'를 유지한다. 단독다가구의 경우, '대지면적'을 다른 변수의 '전용면적'으로 한다. 단독다가구에 '본번', '부번' 변수는 없지만 유지한다.
- 연면적, 대지면적의 계산이나 둘 중 무엇을 다른 변수의 전용면적으로 해야할지 고민해봤는데 모르겠어서 임의로 설정했습니다.
- 또한 층 계산도 용적률과 건폐율이 함께 필요한 것 같아 층 변수를 임의로 삭제했습니다.
- 이외의 층 변수는 단독다가구의 경우 계산이 불가능해 삭제했습니다.

In [16]:
아파트_매매 = 아파트_매매.drop(['층'], axis=1)
연립다세대_매매 = 연립다세대_매매.drop(['대지권면적(㎡)','층'], axis=1)
단독다가구_매매 = 단독다가구_매매.drop(['주택유형','도로조건','연면적(㎡)'], axis=1)
오피스텔_매매 = 오피스텔_매매.drop(['층'], axis=1)

In [17]:
print(list(아파트_매매.columns))
print(list(연립다세대_매매.columns))
print(list(단독다가구_매매.columns))
print(list(오피스텔_매매.columns))

['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전용면적(㎡)', '거래금액(만원)', '건축년도']
['계약년월', '계약일', '시군구', '대지면적(㎡)', '거래금액(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '건축년도']


'계약년월'과 '계약일'을 합쳐 '계약날짜' 변수를 생성한다.

In [18]:
아파트_매매.describe()

,계약년월,계약일,부번,전용면적(㎡),거래금액(만원),건축년도
count,21140.000000,21140.000000,21140.000000,21140.000000,21140.000000,21140.000000
mean,201880.225402,15.359508,2.563198,78.853422,36459.012252,2012.059697
std,114.380451,8.767405,12.020451,22.279039,19863.684392,6.339873
min,201701.000000,1.000000,0.000000,12.179900,1800.000000,1980.000000
25%,201802.000000,8.000000,0.000000,59.972000,24500.000000,2012.000000
50%,201910.000000,15.000000,0.000000,84.857000,33950.000000,2014.000000
75%,202002.000000,23.000000,0.000000,84.990000,46700.000000,2015.000000
max,202010.000000,31.000000,173.000000,209.185200,157000.000000,2020.000000


In [19]:
연립다세대_매매.describe()

,계약년월,계약일,본번,부번,전용면적(㎡),건축년도
count,529.000000,529.000000,529.000000,529.000000,529.000000,529.000000
mean,201851.559546,14.551985,262.151229,11.368620,43.807704,2001.858223
std,127.091431,8.554285,305.365559,49.316025,21.241627,12.421295
min,201701.000000,1.000000,1.000000,0.000000,12.828000,1982.000000
25%,201708.000000,8.000000,112.000000,0.000000,21.634900,1991.000000
50%,201811.000000,14.000000,144.000000,3.000000,48.600000,1996.000000
75%,202005.000000,20.000000,340.000000,10.000000,59.490000,2015.000000
max,202010.000000,31.000000,4121.000000,1002.000000,106.030000,2020.000000


In [20]:
단독다가구_매매.describe()

,계약년월,계약일,대지면적(㎡),거래금액(만원),건축년도
count,997.000000,997.000000,997.000000,997.000000,993.000000
mean,201844.976931,15.790371,419.502979,37645.601805,1991.260826
std,116.113565,8.916344,289.668247,32757.717762,23.301829
min,201701.000000,1.000000,45.000000,800.000000,1900.000000
25%,201710.000000,8.000000,225.000000,15000.000000,1980.000000
50%,201810.000000,16.000000,357.000000,26500.000000,1996.000000
75%,201912.000000,24.000000,532.000000,50500.000000,2011.000000
max,202010.000000,31.000000,2820.000000,350000.000000,2020.000000


In [21]:
오피스텔_매매.describe()

,계약년월,계약일,본번,부번,전용면적(㎡),거래금액(만원),건축년도
count,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000
mean,201906.231343,15.764925,61.492537,0.652985,25.819254,9337.488806,2013.992537
std,117.725664,8.986505,161.745639,1.658852,6.668030,2228.713899,0.457910
min,201701.000000,1.000000,0.000000,0.000000,21.220000,4500.000000,2013.000000
25%,201807.000000,8.000000,0.000000,0.000000,22.420000,7922.500000,2014.000000
50%,202001.000000,16.000000,0.000000,0.000000,23.740000,8500.000000,2014.000000
75%,202007.000000,24.000000,0.000000,0.000000,25.627500,10000.000000,2014.000000
max,202010.000000,31.000000,713.000000,5.000000,83.010000,19000.000000,2019.000000


이상치가 없으니 그대로 진행한다.

In [22]:
def create_계약날짜(data):
    data['계약년월'] = data['계약년월'].astype(str)
    data['계약일'] = data['계약일'].astype('int64').apply(lambda x: "{:0>2d}".format(x))
    data['계약날짜'] = data[['계약년월','계약일']].apply(lambda x: ''.join(x), axis=1)

    data = data.drop(['계약년월','계약일'], axis=1)
    return data

아파트_매매 = create_계약날짜(아파트_매매)
연립다세대_매매 = create_계약날짜(연립다세대_매매)
단독다가구_매매 = create_계약날짜(단독다가구_매매)
오피스텔_매매 = create_계약날짜(오피스텔_매매)

In [23]:
아파트_매매.head()

,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),건축년도,계약날짜
0,세종특별자치시 보람동,628,2,스타힐타워Ⅰ(C2-3),16.9849,8600,2016,20170101
1,세종특별자치시 종촌동,683,0,가재마을2단지(호반베르디움3차),59.8652,27300,2014,20170101
2,세종특별자치시 고운동,1393,0,가락마을15단지(중흥파크뷰),84.9894,32000,2015,20170102
3,세종특별자치시 고운동,1706,0,가락마을8단지(고운뜰아파트),84.9800,30500,2015,20170102
4,세종특별자치시 아름동,1284,0,범지기마을7단지 호반에코시티,84.9280,31300,2014,20170102


'시군구'와 '단지명'/'건물명'을 합쳐 '건물위치' 변수를 생성한다.

In [24]:
단독다가구_매매.head()

,시군구,대지면적(㎡),거래금액(만원),건축년도,계약날짜
0,세종특별자치시 전의면 다방리,294.0,2000,1991.0,20170102
1,세종특별자치시 조치원읍 서창리,415.0,54500,1997.0,20170104
2,세종특별자치시 조치원읍 죽림리,175.0,16000,1983.0,20170104
3,세종특별자치시 금남면 용담리,537.0,26000,1977.0,20170106
4,세종특별자치시 조치원읍 서창리,222.7,53700,2011.0,20170106


In [25]:
def create_건물위치(data, building):
    data['건물위치'] = data[['시군구',building]].apply(lambda x: ' '.join(x), axis=1)

    data = data.drop(['시군구',building], axis=1)
    return data

아파트_매매 = create_건물위치(아파트_매매, '단지명')
연립다세대_매매 = create_건물위치(연립다세대_매매, '건물명')
오피스텔_매매 = create_건물위치(오피스텔_매매, '단지명')
단독다가구_매매 = 단독다가구_매매.rename(columns={'시군구':'건물위치'})

In [26]:
아파트_매매.head(2)

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치
0,628,2,16.9849,8600,2016,20170101,세종특별자치시 보람동 스타힐타워Ⅰ(C2-3)
1,683,0,59.8652,27300,2014,20170101,세종특별자치시 종촌동 가재마을2단지(호반베르디움3차)


In [27]:
연립다세대_매매.head(2)

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치
0,132,7,62.08,"4,700",1983,20170103,세종특별자치시 조치원읍 원리 고려1동
1,167,6,64.23,"6,500",1991,20170113,세종특별자치시 조치원읍 침산리 용성빌라


In [28]:
오피스텔_매매.head(2)

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치
0,0,0,23.74,9000,2014,20170104,세종특별자치시 어진동 푸르지오시티
1,0,0,23.74,9500,2014,20170106,세종특별자치시 어진동 푸르지오시티


In [29]:
단독다가구_매매 = 단독다가구_매매.rename(columns={'대지면적(㎡)':'전용면적(㎡)'})
단독다가구_매매.head(2)

,건물위치,전용면적(㎡),거래금액(만원),건축년도,계약날짜
0,세종특별자치시 전의면 다방리,294.0,2000,1991.0,20170102
1,세종특별자치시 조치원읍 서창리,415.0,54500,1997.0,20170104


매매 데이터를 한 데이터로 합치기 위해 '주택유형' 변수를 추가한다. 
- 1: 아파트
- 2: 연립다세대
- 3: 단독다가구
- 4: 오피스텔

In [30]:
for i, data in enumerate([아파트_매매, 연립다세대_매매, 단독다가구_매매, 오피스텔_매매]):
    data['주택유형'] = i+1

아파트_매매.head(2)

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치,주택유형
0,628,2,16.9849,8600,2016,20170101,세종특별자치시 보람동 스타힐타워Ⅰ(C2-3),1
1,683,0,59.8652,27300,2014,20170101,세종특별자치시 종촌동 가재마을2단지(호반베르디움3차),1


In [31]:
오피스텔_매매.head(2)

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치,주택유형
0,0,0,23.74,9000,2014,20170104,세종특별자치시 어진동 푸르지오시티,4
1,0,0,23.74,9500,2014,20170106,세종특별자치시 어진동 푸르지오시티,4


In [32]:
매매_실거래가 = pd.concat([아파트_매매, 연립다세대_매매, 단독다가구_매매, 오피스텔_매매])
매매_실거래가.tail()

,본번,부번,전용면적(㎡),거래금액(만원),건축년도,계약날짜,건물위치,주택유형
263,0,0.0,24.22,9000,2014.0,20200923,세종특별자치시 어진동 푸르지오시티,4
264,0,0.0,21.92,7700,2014.0,20200924,세종특별자치시 어진동 2차푸르지오시티,4
265,0,0.0,23.74,8200,2014.0,20200928,세종특별자치시 어진동 푸르지오시티,4
266,0,0.0,23.74,8050,2014.0,20201005,세종특별자치시 어진동 푸르지오시티,4
267,0,0.0,21.92,7500,2014.0,20201006,세종특별자치시 어진동 2차푸르지오시티,4


### 2.2. 전월세 실거래가

In [33]:
print(list(아파트_전월세.columns))
print(list(연립다세대_전월세.columns))
print(list(단독다가구_전월세.columns))
print(list(오피스텔_전월세.columns))

['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '도로조건', '계약면적(㎡)', '전월세구분', '보증금(만원)', '월세(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']


In [34]:
아파트_전월세 = 아파트_전월세.drop(['층'], axis=1)
연립다세대_전월세 = 연립다세대_전월세.drop(['층'], axis=1)
단독다가구_전월세 = 단독다가구_전월세.drop(['도로조건'], axis=1)
오피스텔_전월세 = 오피스텔_전월세.drop(['층'], axis=1)

In [35]:
아파트_전월세.isnull().sum()

계약년월       0
계약일        0
시군구        0
본번         0
부번         0
단지명        0
전월세구분      0
전용면적(㎡)    0
보증금(만원)    0
월세(만원)     0
건축년도       0
dtype: int64

In [36]:
연립다세대_전월세.isnull().sum()

계약년월       0
계약일        0
시군구        0
본번         0
부번         0
건물명        0
전월세구분      0
전용면적(㎡)    0
보증금(만원)    0
월세(만원)     0
건축년도       0
dtype: int64

In [37]:
단독다가구_전월세.isnull().sum()

계약년월         0
계약일          0
시군구          0
계약면적(㎡)      0
전월세구분        0
보증금(만원)      0
월세(만원)       0
건축년도       210
dtype: int64

In [38]:
오피스텔_전월세.isnull().sum()

계약년월       0
계약일        0
시군구        0
본번         0
부번         0
단지명        0
전월세구분      0
전용면적(㎡)    0
보증금(만원)    0
월세(만원)     0
건축년도       0
dtype: int64

In [39]:
아파트_전월세 = create_계약날짜(아파트_전월세)
연립다세대_전월세 = create_계약날짜(연립다세대_전월세)
단독다가구_전월세 = create_계약날짜(단독다가구_전월세)
오피스텔_전월세 = create_계약날짜(오피스텔_전월세)

In [40]:
아파트_전월세 = create_건물위치(아파트_전월세, '단지명')
연립다세대_전월세 = create_건물위치(연립다세대_전월세, '건물명')
오피스텔_전월세 = create_건물위치(오피스텔_전월세, '단지명')
단독다가구_전월세 = 단독다가구_전월세.rename(columns={'시군구':'건물위치', '계약면적(㎡)':'전용면적(㎡)'})

In [41]:
print(list(아파트_전월세.columns))
print(list(연립다세대_전월세.columns))
print(list(단독다가구_전월세.columns))
print(list(오피스텔_전월세.columns))

['본번', '부번', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '건축년도', '계약날짜', '건물위치']
['본번', '부번', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '건축년도', '계약날짜', '건물위치']
['건물위치', '전용면적(㎡)', '전월세구분', '보증금(만원)', '월세(만원)', '건축년도', '계약날짜']
['본번', '부번', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '건축년도', '계약날짜', '건물위치']


In [42]:
for i, data in enumerate([아파트_전월세, 연립다세대_전월세, 단독다가구_전월세, 오피스텔_전월세]):
    data['주택유형'] = i+1

아파트_전월세.head(2)

,본번,부번,전월세구분,전용면적(㎡),보증금(만원),월세(만원),건축년도,계약날짜,건물위치,주택유형
0,1409,0,전세,59.9907,16000,0,2016,20170101,세종특별자치시 고운동 가락마을11단지(중흥에듀카운티),1
1,35,3,전세,28.1680,2800,0,2002,20170101,세종특별자치시 조치원읍 신안리 유니빌,1


In [43]:
전월세_실거래가 = pd.concat([아파트_전월세, 연립다세대_전월세, 단독다가구_전월세, 오피스텔_전월세])
전월세_실거래가.tail(3)

,본번,부번,전월세구분,전용면적(㎡),보증금(만원),월세(만원),건축년도,계약날짜,건물위치,주택유형
1208,246,5.0,월세,24.03,500,37,2014.0,20201005,세종특별자치시 도담동 세종한신휴시티,4
1209,246,5.0,월세,24.03,500,40,2014.0,20201006,세종특별자치시 도담동 세종한신휴시티,4
1210,713,0.0,월세,34.15,3000,35,2013.0,20201021,세종특별자치시 나성동 세진이너스빌,4


# geojson test

In [53]:
도로명주소_건물 = gpd.read_file('/content/drive/Shareddrives/세종시_프로젝트/data/23.세종시_도로명주소_건물.geojson')
법정경계_읍면동 = gpd.read_file('/content/drive/Shareddrives/세종시_프로젝트/data/31.세종시_법정경계(읍면동).geojson')
도로명주소_건물.head(2)

,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry
0,10299,None,None,0,55874,350,1,177,4,0,"MULTIPOLYGON (((127.19039 36.49291, 127.19052 ..."
1,10299,None,None,0,55795,340,1,42,23,0,"MULTIPOLYGON (((127.29546 36.42295, 127.29564 ..."


In [45]:
도로명주소_건물['EMD_CD'].isnull().sum()

0

In [46]:
도로명주소_건물['EMD_CD'].unique()

array(['350', '340', '107', '110', '114', '113', '320', '118', '101',
       '102', '103', '104', '380', '310', '330', '108', '111', '112',
       '360', '250', '370', '390', '115', '117', '106', '105', '109',
       '116'], dtype=object)

In [47]:
geo_data = '/content/drive/Shareddrives/세종시_프로젝트/data/23.세종시_도로명주소_건물.geojson'
df = 표제부
df = df.astype({'법정동코드':str})
df['법정동'] = df['법정동코드'].str.slice(0,3)

In [48]:
df.head(2)

,순번,대지위치,시군구코드,법정동코드,대지구분코드,대장구분코드명,건물명,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,법정동
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,일반,NaN,135.24,벽돌구조,단독주택,0,1977.0,101
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,일반,NaN,576,강파이프구조,동.식물관련시설,0,1999.0,101


In [49]:
import fiona
ddd = fiona.open('/content/drive/Shareddrives/세종시_프로젝트/data/23.세종시_도로명주소_건물.geojson')
ddd[0]

{'geometry': {'coordinates': [[[(127.19038809421353, 36.49291470796905),
     (127.19052390695718, 36.49276789604461),
     (127.19084207287948, 36.4929477056725),
     (127.19074430936344, 36.49306220630684),
     (127.1905643165508, 36.49295987387606),
     (127.19052810455064, 36.493000978366034),
     (127.19038809421353, 36.49291470796905)]]],
  'type': 'MultiPolygon'},
 'id': '0',
 'properties': OrderedDict([('BDTYP_CD', '10299'),
              ('BULD_NM', None),
              ('BULD_NM_DC', None),
              ('BULD_SE_CD', '0'),
              ('BUL_MAN_NO', 55874),
              ('EMD_CD', '350'),
              ('GRO_FLO_CO', 1),
              ('LNBR_MNNM', 177),
              ('LNBR_SLNO', 4),
              ('UND_FLO_CO', 0)]),
 'type': 'Feature'}

In [54]:
# 세종시 중심부의 위도, 경도 입니다.
center = [36.57204094280581, 127.2666604789283]

# 맵이 center 에 위치하고, zoom 레벨은 12로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center, zoom_start=11)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=법정경계_읍면동,
    data=df,
    columns=('법정동','연면적(㎡)'),
    #key_on='EMD_CD',
    fill_color='BuPu',
    legend_name='연면적',
).add_to(m)

# 맵 m을 출력합니다.
m